In [ ]:
import math
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import scipy.stats
import time
import seaborn as sns

In [ ]:
#Define the function b
def b(theta, a, x):
  if theta == 1:
    y = np.array([1 - a[0]*x[0] - x[0]**3, 1 - a[1]*x[1] - x[1]**3, 1 - a[2]*x[2] - x[2]**3])
  if theta == 2:
    y = np.array([2 - a[0]*x[0] - x[0]**3, 2 - a[1]*x[1] - x[1]**3, 2 - a[2]*x[2] - x[2]**3])
  return y

In [ ]:
#Define the function sigma
def sigma(theta, B, x):
  if theta == 1:
    y = np.array([B[0]*x[0]**2, B[1]*x[1]**2, B[2]*x[2]**2])
  if theta == 2:
    y = np.array([B[0]*x[2]**2, B[1]*x[0]**2, B[2]*x[1]**2])
  return y

In [ ]:
#Define the function gamma
def c(theta, g, x):
  if theta == 1:
    y = np.array([g[0]*x[0], g[1]*(x[1] + math.sin(x[2])), g[2]*math.cos(x[2])])
  if theta == 2:
    y = np.array([g[0]*math.cos(x[2]), g[1]*x[0], g[2]*(x[1] + math.sin(x[2]))])
  return y

In [ ]:
#Define the tamed function sigma
def smdelta(theta, B, x, Dt):
  tg = 1 + math.sqrt(Dt)*np.linalg.norm(sigma(theta, B, x))
  y = sigma(theta, B, x)/tg
  return y

In [ ]:
#Define the tamed function gamma
def cdelta(theta, B, g, x, Dt):
  tg = 1 + math.sqrt(Dt)*np.linalg.norm(c(theta, g, x))*(1 + np.linalg.norm(b(theta, B, x)))
  y = c(theta, g, x)/tg
  return y

In [ ]:
#Define the step function h
def h(x,ell,p0):
  u = (1 + np.linalg.norm(b(1,a,x))**2 + np.linalg.norm(b(2,a,x))**2 + np.linalg.norm(sigma(1,B,x)) + np.linalg.norm(sigma(2,B,x)) + np.linalg.norm(x)**ell)**2 + (np.linalg.norm(c(1,g,x)) + np.linalg.norm(c(2,g,x)))**p0
  y = 1/u
  return y

In [ ]:
def hdelta(x,ell, p0, delta):
  y= h(x,ell, p0)*delta
  return y

In [ ]:
def  levyZ(t, ld, sm):
# generate levyZ(t) = sum_{i=1}^N(t) Y_i
# where N is poisson process with intensity lambda ld
# and Y_i has normal distribution with mean zero and variance sigma^2
  if t==0:
    y = 0
  if t >0:
    r = poisson.rvs(t*ld, size=3)
    y = sm * np.sqrt(r) * np.random.normal(0, 1, size = 3)
  return y

In [ ]:
def levyGM(t, gm, ld):
# generate levyGM(t) bilateral Gamma process
  if t==0:
    y = 0
  if t >0:
    y = np.random.gamma(t*gm, ld, 3) - np.random.gamma(t*gm, ld, 3)
  return y

In [ ]:
#Define the trial function phi1, phi2, and phi3
def phi1(X):
  return X[0] + X[1] + X[2]
def phi2(X):
  return (X[0])**2 + (X[1])**2 +(X[2])**2
def phi3(X):
  return np.abs(phi1(X)/(np.abs(phi1(X)) + 1))

In [ ]:
x0 = np.array([0, 0, 0])
T = 10
ld = 10
sm = 0.4
ell = 2
p0 = 10
a = np.array([1, 1, 1])
B = np.array([0.1, 0.1, 0.1])
g = np.array([0.2, 0.2, 0.2])

In [ ]:
def twolevel(l, T):
  t = 0
  t_c = 0
  t_f = 0
  theta_old_c = 1
  theta_c = 1
  theta_old_f = 1
  theta_f = 1
  theta = 1
  h_c = 0
  h_f = 0
  dW_c = np.array([0, 0, 0])
  dW_f = np.array([0, 0, 0])
  dZ_c = np.array([0, 0, 0])
  dZ_f = np.array([0, 0, 0])
  n_c=2**l
  n_f=2**(l+1)
  delta_c=1/n_c
  delta_f=1/n_f
  X_c = x0
  X_f = x0
  while t<T:
      #rand('normal')
      t_old = t
      theta_old_c = theta_c
      theta_old_f = theta_f
      t = min(t_c,t_f)
      dW = math.sqrt(t-t_old)*np.random.normal(0, 1, size = 3)
      dZ = levyGM(t-t_old, ld, sm)
      dW_c = dW_c + dW
      dW_f = dW_f + dW
      dZ_c = dZ_c + dZ
      dZ_f = dZ_f + dZ
      V = np.random.uniform(0,1)
      if theta == 1:
        if V < 0.1*(t-t_old):
          theta = 2
      else:
        if V < 0.2*(t-t_old):
          theta = 1
      if t==t_c:
        X_c = X_c + b(theta_old_c, a, X_c)*h_c + smdelta(theta_old_c, B, X_c,delta_c)*dW_c + cdelta(theta_old_c, B, g, X_c, delta_c)* dZ_c  #update coarse path X_c using h_c and W_c
        h_c =  h(X_c, ell, p0)*delta_c    #compute new adapted coarse path timestep
        h_c = min(h_c,T-t_c)   #Check the time point that is nearest to T
        t_c = t_c+h_c        #update t_c
        dW_c = np.array([0, 0, 0])
        dZ_c = np.array([0, 0, 0])
        theta_c = theta
      if t==t_f:
        X_f = X_f + b(theta_old_f, a, X_f)*h_f + smdelta(theta_old_f, B, X_f, delta_f)*dW_f + cdelta(theta_old_f, B, g, X_f, delta_f)*dZ_f #update fine path X_f using h_f and W_f
        h_f = h(X_f, ell, p0)*delta_f   #compute new adapted fine path timestep
        h_f = min(h_f,T-t_f)    #Check the time point that is nearest to T
        t_f = t_f+h_f     #update t_f
        dW_f = np.array([0, 0, 0])
        dZ_f = np.array([0, 0, 0])
        theta_f = theta
  kq3hamphi = np.array([phi1(X_f)-phi1(X_c), phi2(X_f)-phi2(X_c), phi3(X_f)-phi3(X_c)])
  return kq3hamphi

In [ ]:
def level0(T):
  t = 0
   # update h_f
  theta = 1
  dW = np.array([0, 0, 0])
  dZ = np.array([0, 0, 0])
  X_f = x0
  while t<T:
      h0 = h(X_f, ell, p0)
      dW = math.sqrt(h0)*np.random.normal(0, 1, size = 3)
      dZ = levyGM(h0, ld, sm)
      X_f = X_f + b(theta, a, X_f)*h0 + smdelta(theta, B, X_f, 1)*dW + cdelta(theta, B, g, X_f, 1)*dZ #update X_f
      V = np.random.uniform(0,1)
      if theta == 1:
        if V < 0.1*h0:
          theta = 2
      else:
        if V < 0.2*h0:
          theta = 1
      t = t + h0     #update t_f
  kq3hamphi = np.array([phi1(X_f), phi2(X_f), phi3(X_f)])
  return kq3hamphi

In [ ]:
T = 20
L = 7
N_l = np.array([0]*(L+1))
N_l[0] = 2**10
for l in range(L):
  N_l[l+1] = (N_l[l]/2)*math.sqrt((l+1)/(l+2))
N_l = np.floor(N_l) + 1
print(N_l)

In [ ]:
def Y_hat(N_l_e, L_e, T_e):
  sum_level_0 = np.array([0]*3)
  for i in range(int(N_l_e[0])):
    sum_level_0 = sum_level_0 + level0(T_e)

  sum_two_level = np.array([[0]*3]*L)
  sum_all_level = np.array([0]*3)
  for i in range(L_e):
    for j in range(int(N_l_e[i])):
      sum_two_level[i] = sum_two_level[i] + twolevel(i,T_e)
    sum_all_level = sum_all_level + 1/N_l_e[i]*sum_two_level[i]
  return 1/N_l_e[0]*(sum_level_0) + sum_all_level

In [ ]:
def point_T_e(k):
  T_e_0 = [0]*(k+1)
  for i in range(k+1):
    T_e_0[i] = 3 + 2*i*math.log(2)
  return T_e_0

In [ ]:
def point_L_e(k):
  L_e_0 = [0]*(k+1)
  for i in range(k+1):
    L_e_0[i] = 3 + i
  return L_e_0

In [ ]:
def point_N_l_e(k):
  N_l_e_0 = [0]*(k+1)
  for i in range(k+1):
    N_l_e_0[i] = [0]*point_L_e(k)[i]
  for i in range(k+1):
    N_l_e_0[i][0] = 32*(4**i)
    for j in range(len(N_l_e_0[i]) - 1):
      N_l_e_0[i][j+1] = (N_l_e_0[i][j]/2)*math.sqrt((l+1)/(l+2))
    N_l_e_0[i] = np.floor(N_l_e_0[i]) + 1
  return N_l_e_0

In [ ]:
#Testing, sample = 10
start_time = time.time()
Y_hat_phi1 = []
Y_hat_phi2 = []
Y_hat_phi3 = []
for i in range(10):
  a = Y_hat(point_N_l_e(4)[0], point_L_e(4)[0], point_T_e(4)[0])
  print(a)
  Y_hat_phi1.append(a[0])
  Y_hat_phi2.append(a[1])
  Y_hat_phi3.append(a[2])
end_time = time.time()
print("Computational cost:")
print(end_time - start_time)

In [ ]:
#Number of sample for epsilon/16
st = 2

In [ ]:
#Draw box plots of Y_hat, epsilon = epsilon_0
Y_hat_phi1_epsilon0 = []
Y_hat_phi2_epsilon0 = []
Y_hat_phi3_epsilon0 = []
start_time = time.time()
for i in range(st*(2**4)):
  a = Y_hat(point_N_l_e(4)[0], point_L_e(4)[0], point_T_e(4)[0])
  print(a)
  Y_hat_phi1_epsilon0.append(a[0])
  Y_hat_phi2_epsilon0.append(a[1])
  Y_hat_phi3_epsilon0.append(a[2])
end_time = time.time()
print("Computational cost:")
print(end_time - start_time)

In [ ]:
#Draw box plots of Y_hat, epsilon = epsilon_0/2
Y_hat_phi1_epsilon1 = []
Y_hat_phi2_epsilon1 = []
Y_hat_phi3_epsilon1 = []
start_time = time.time()
for i in range(st*(2**3)):
  a = Y_hat(point_N_l_e(4)[1], point_L_e(4)[1], point_T_e(4)[1])
  print(a)
  Y_hat_phi1_epsilon1.append(a[0])
  Y_hat_phi2_epsilon1.append(a[1])
  Y_hat_phi3_epsilon1.append(a[2])
end_time = time.time()
print("Computational cost:")
print(end_time - start_time)

In [ ]:
#Draw box plots of Y_hat, epsilon = epsilon_0/(2^2)
Y_hat_phi1_epsilon2 = []
Y_hat_phi2_epsilon2 = []
Y_hat_phi3_epsilon2 = []
start_time = time.time()
for i in range(st*(2**2)):
  a = Y_hat(point_N_l_e(4)[2], point_L_e(4)[2], point_T_e(4)[2])
  print(a)
  Y_hat_phi1_epsilon2.append(a[0])
  Y_hat_phi2_epsilon2.append(a[1])
  Y_hat_phi3_epsilon2.append(a[2])
end_time = time.time()
print("Computational cost:")
print(end_time - start_time)

In [ ]:
#Draw box plots of Y_hat, epsilon = epsilon_0/(2^3)
Y_hat_phi1_epsilon3 = []
Y_hat_phi2_epsilon3 = []
Y_hat_phi3_epsilon3 = []
start_time = time.time()
for i in range(st*2):
  a = Y_hat(point_N_l_e(4)[3], point_L_e(4)[3], point_T_e(4)[3])
  print(a)
  Y_hat_phi1_epsilon3.append(a[0])
  Y_hat_phi2_epsilon3.append(a[1])
  Y_hat_phi3_epsilon3.append(a[2])
end_time = time.time()
print("Computational cost:")
print(end_time - start_time)

In [ ]:
#Draw box plots of Y_hat, epsilon = epsilon_0/(2^4)
Y_hat_phi1_epsilon4 = []
Y_hat_phi2_epsilon4 = []
Y_hat_phi3_epsilon4 = []
start_time = time.time()
for i in range(st):
  a = Y_hat(point_N_l_e(4)[4], point_L_e(4)[4], point_T_e(4)[4])
  print(a)
  Y_hat_phi1_epsilon4.append(a[0])
  Y_hat_phi2_epsilon4.append(a[1])
  Y_hat_phi3_epsilon4.append(a[2])
end_time = time.time()
print("Computational cost:")
print(end_time - start_time)

In [ ]:
#Use tamed-adaptive EM scheme to approximate the solution of the SDE, then draw the regression line to see the convergence rate
def kq(L, nMC):
    SS = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
    for l in range(2,L+1):
        saiso=0
        for iMC in range(1, nMC+1):
            t = 0
            t_c = 0
            t_f = 0
            theta_old_c = 1
            theta_c = 1
            theta_old_f = 1
            theta_f = 1
            theta = 1
            h_c = 0
            h_f = 0
            dW_c = np.array([0, 0, 0])
            dW_f = np.array([0, 0, 0])
            dZ_c = np.array([0, 0, 0])
            dZ_f = np.array([0, 0, 0])
            n_c=2**l
            n_f=2**(l+1)
            delta_c=1/n_c
            delta_f=1/n_f
            X_c = x0
            X_f = x0
            while t<T:
                #rand('normal')
                t_old = t
                theta_old_c = theta_c
                theta_old_f = theta_f
                t = min(t_c,t_f)
                dW = math.sqrt(t-t_old)*np.random.normal(0, 1, size = 3)
                dZ = levyGM(t-t_old, ld, sm)
                dW_c = dW_c + dW
                dW_f = dW_f + dW
                dZ_c = dZ_c + dZ
                dZ_f = dZ_f + dZ
                V = np.random.uniform(0,1)
                if theta == 1:
                  if V < 0.1*(t-t_old):
                    theta = 2
                else:
                  if V < 0.2*(t-t_old):
                    theta = 1
                if t==t_c:
                  X_c = X_c + b(theta_old_c, a, X_c)*h_c + smdelta(theta_old_c, B, X_c,delta_c)*dW_c + cdelta(theta_old_c, B, g, X_c, delta_c)* dZ_c  #update X_c
                  h_c =  h(X_c, ell, p0)*delta_c # hdelta(X_c,delta_c)   # update h_c
                  h_c = min(h_c,T-t_c)   #kiem tra moc thoi diem gan T nhat
                  t_c = t_c+h_c        #update t_c
                  dW_c = np.array([0, 0, 0])           #neu nhay thi bang 0
                  dZ_c = np.array([0, 0, 0])
                  theta_c = theta
                if t==t_f:
                  X_f = X_f + b(theta_old_f, a, X_f)*h_f + smdelta(theta_old_f, B, X_f, delta_f)*dW_f + cdelta(theta_old_f, B, g, X_f, delta_f)*dZ_f #update X_f
                  h_f = h(X_f, ell, p0)*delta_f   # update h_f
                  h_f = min(h_f,T-t_f)    #kiem tra moc thoi diem gan T nhat
                  t_f = t_f+h_f     #update t_f
                  dW_f = np.array([0, 0, 0])         #neu nhay thi bang 0
                  dZ_f = np.array([0, 0, 0])
                  theta_f = theta
            saiso=saiso + (np.linalg.norm(X_c-X_f))**2
        Esaiso= math.log(saiso/nMC)/math.log(2)
        SS[l-2] = Esaiso
        print(Esaiso)
    v = np.array([2, 3, 4, 5, 6]).reshape((-1, 1))
    model = LinearRegression().fit(v, SS)
    r_sq = model.score(v, SS)
    print(f"coefficient of determination: {r_sq}")
    print(f"intercept: {model.intercept_}")
    print(f"slope: {model.coef_}")
    return None

In [ ]:
#Computational cost
computational_cost = [55.05143332481384, 179.55675601959229, 528.8837549686432, 1483.4553360939026, 3898.9569458961487]
log2_computational_cost = np.log(computational_cost)/math.log(2)
log2_computational_cost

In [ ]:
#Regression line of RMS and computational cost
log_2_epsilon = np.array([-1, -2, -3, -4, -5]).reshape((-1, 1))
model = LinearRegression().fit(log_2_epsilon, log2_computational_cost)
r_sq = model.score(log_2_epsilon, log2_computational_cost)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

plt.scatter(log_2_epsilon, log2_computational_cost)

# fitting a linear regression line
m, b = np.polyfit([-1, -2, -3, -4, -5], log2_computational_cost, 1)

# adding the regression line to the scatter plot
plt.xlabel("1")
plt.ylabel("2")
plt.plot(log_2_epsilon, m*log_2_epsilon + b)